In [2]:
# Data
using NamedArrays

nodes = [:S, :K, :P, :gamma, :L, :B, :O, :beta, :C, :M, :R, :alpha]
us = [:K, :L, :C]
producers = [us..., :gamma, :beta, :alpha]
consumers = [:S, :K, :P, :L, :B, :O, :C, :M, :R]

arcs = [(:K, :P), (:S, :P), (:P, :L), (:P, :gamma), 
    (:gamma, :B), (:gamma, :L), (:gamma, :R), (:L, :B), (:B, :O),
    (:O, :beta), (:beta, :R), (:beta, :C), (:beta, :M),
    (:alpha, :gamma), (:alpha, :S), (:alpha, :R)]
arcs = [arcs; [(j, i) for (i, j) in arcs]]

dist_vec = [1, 2, 6, 3, 4, 5, 7, 5, 3, 4, 2, 2, 3, 5, 5, 5]*100
distance = Dict(zip(arcs, [dist_vec; dist_vec]))

days = [1:7;]

loss = 0.0005

matrix = zeros(length(nodes), length(days))
demand = NamedArray(matrix, (nodes, days), ("Consumer","Day"))
demand[:S, :] = [40.8, 100, 230.5, 150.8, 74.2, 97.3, 65.4]
demand[:P, :] = [37.8, 30.3, 28.8, 20.5, 37, 45, 29.6]
demand[:K, :] = repeat([27.3], length(days));
demand[:L, :] = [30.1, 28.8, 27.3, 55.2, 64, 42, 36.6]
demand[:C, :] = [30, 35.5, 31.2, 45.5, 50, 43.1, 38.4]
demand[:B, :] = [30.1, 31.5, 30.1, 35.1, 34.3, 39, 33.2]
demand[:O, :] = [23.7, 28.3, 16.4, 36.7, 20, 17.3, 11.1]
demand[:M, :] = [27.3, 21.9, 24.6, 28.9, 26, 30.7, 23.8]
demand[:R, :] = [19.2, 20.5, 20.5, 20, 18.4, 21.3, 20.5]

matrix = zeros(length(nodes), length(days))
supply = NamedArray(matrix, (nodes, days), ("Producer","Day"))
supply[:alpha, :] = [600, 300, 700, 0, 100, 0, 350]
supply[:beta, :] = [500, 200, 900, 0, 50, 0, 300]
supply[:L, :] = [150, 200, 150, 305, 200, 30, 500]
supply[:K, :] = [200, 150, 100, 600, 150, 40, 200]
supply[:C, :] = [150, 230, 180, 120, 40, 20, 140]
supply[:gamma, :] .= 10000;

# punishment for using gamma
µ = [0.01, 0.07, 0.22;];

# battery capacities
cap = Dict(zip(nodes, zeros(length(nodes))))
for c in consumers
    cap[c] = 20;
end
cap[:S] = 100;

# battery efficiency
eff_in = 0.6;
eff_out = 0.8;

# cost per kWh lost
cost = 1;

# dirty power penalty
penalty = 10;

# price of energy sold to the grid
price = 1.5;

# cycle prevention coefficient
cp = 0.16;

In [3]:
using GraphPlot, LightGraphs, Colors

# Plots the graph of the flows
# flow is a dictionary of arcs -> flows
# Green nodes represents energy-giving nodes, Red nodes represents energy-taking nodes, Gray nodes are neutral nodes.
# Green edges means energy flows there
# Labels appear at the destination of nodes
function plot_flow(flow)
    
N = length(nodes)
E = length(arcs)
nodemap = Dict(zip(nodes, 1:N))
g = SimpleDiGraph(N)
for (i,j) in arcs
    add_edge!(g, nodemap[i], nodemap[j])
end
                                                            
inflows = Dict(zip(nodes, [sum(flow[(i,j)] for (i,j) in arcs if j == k) for k in nodes]))
outflows = Dict(zip(nodes, [sum(flow[(i,j)] for (i,j) in arcs if i == k) for k in nodes]))
                                                                                        
# vector of net outflows for each node
net_outflows = [outflows[k] - inflows[k] for k in nodes]

# color nodes in proportion to their net outflows                                                            
l, u = min(net_outflows...), max(net_outflows...) # lower and upper bounds on the colors
node_colors = [(c > 0 ? RGBA(0.5*(1-c/u), 0.5*(1+c/u), 0.5*(1-c/u), 1) 
        : RGBA(0.5*(1+c/l), 0.5*(1-c/l), 0.5*(1-c/l), 1)) for c in net_outflows]

# sort arcs by their tuple of numerical node indices
arcmap = sort(arcs, by=(n -> (nodemap[n[1]], nodemap[n[2]])))

# locations of the nodes, note that y is reversed
locs_x = [0, 0.5, 1, 2, 4, 3.5, 4, 2, 1.5, 3, 1, 0]
locs_y = -[0, -0.5, 0, 1, 0, 2, 3, 3, 4, 4, 3.5, 2]
    
# color the edges in proportion to their flow
maxflow = max([flow[a] for a in arcs]...)
lg = colorant"lightgray" # lightgray
dominant_flows = Dict(zip(arcs, [max(flow[(i,j)], flow[(j,i)]) for (i,j) in arcs]))
edge_colors = [(dominant_flows[a] > 0 ? RGBA(lg.r*(1-dominant_flows[a]/maxflow), 
        dominant_flows[a]*(1-lg.g)/maxflow + lg.g, 
        lg.b*(1-dominant_flows[a]/maxflow), 1) : lg) for a in arcmap]
    
# plot graph
gplot(g, nodelabel=[(n in producers ? String(n)*"*" : n) for n in nodes], locs_x, locs_y,
    edgelabel=[(flow[a] > 0 ? string(round(flow[a],1)) : "") for a in arcmap], 
    arrowlengthfrac=0, edgelabeldistx=0, edgelabeldisty=0,
    nodefillc=node_colors, edgestrokec=edge_colors, NODESIZE=0.05)
                                                            
end
                                                        
# flow = Dict(zip(arcs, [getvalue(x[a, day]) for a in arcs]))

# plot_flow(flow)

plot_flow (generic function with 1 method)

In [65]:
# Model
using Clp, JuMP

# fill an array of length len using non-repeating elements of A
function fillrand(A, len)
    B = copy(A)
    C = Array{typeof(A[1])}(0)
    for i in 1:len
        day = rand(B)
        deleteat!(B, findin(B, day))
        append!(C, day)
    end
    return C
end

# --- solve the model with given parameters ---
# price: how much sharing energy (per kWh) is rewarded
# cost: how much energy losses cost (per kWh)
# penalty: how much dirty power is punished (per kWh)
# cp: how much the flow capacities sum should be punished (per kWh)
# verbose: whether to display large output info
# random_demand: whether to generate randomized demand
# random_supply: whether to generate randomized supply
# seed: random seed to use (default between 1 to 1000000)
# random_dist: what type of random distribution to use (:Gaussian or :Uniform)
# random_std_p: percent (0 to 1) of the mean the random distribution used for its std
# no_wind: either an array of days without wind, or the number of randomly generated no wind days
# no_sun: either an array of days without sun, or the number of randomly generated no sun days
# cp_step: what to increase cp by during each iteration with cycles
# max_iter: maximum number of solves until no cycles
function solve_flows(;price=1, cost=1, penalty=10, cp=10, verbose=true, 
                            random_demand=false, random_supply=false, seed=-1,
                            random_dist=:Gaussian, random_std_p=0.3,
                            no_wind = [], no_sun = [],
                            cp_step = 5, max_iter = 10)
# calculate useful set sizes
N = length(nodes)
T = length(days)
E = length(arcs)
    
if typeof(max_iter) != Int64
    println("Invalid type for max_iter!")
    return
end
    
if max_iter < 1
    println("Invalid value for max_iter (must be >= 1): ", max_iter)
    return
end
    
if typeof(cp_step) != Int64
    println("Invalid type for cp_step!")
    return
end

if cp_step <= 0
    println("Invalid cp_step (must be > 0): ", cp_step)
    return
end    
    
# check if random no wind days
random_no_wind = false
if typeof(no_wind) == Int64
    if !(1 <= no_wind <= length(days))
        println("Error: ", no_wind, " is not a valid number of days!")
        println("Ignoring no wind days...")
    else
        random_no_wind = true
    end
end
    
# check if random no sun days
random_no_sun = false
if typeof(no_sun) == Int64
    if !(0 <= no_sun <= length(days))
        println("Error: ", no_sun, " is not a valid number of days!")
        println("Ignoring no sun days...")
    else
        random_no_sun = true
    end
end
    
# will the model be randomized?
random = random_demand | random_supply | random_no_wind | random_no_sun
    
# if seed is provided, seed the random number generator (RNG), otherwise, generate it with a seed from 1:1000000
if seed >= 0 
    srand(seed); # seed the RNG
    if !random
        println("Seed provided but nothing is being randomized! Oops...")
        println("Using default values for supply and demand.")
    end
else
    seed = rand(1:1000000)
    if random
        println("Seed: ", seed)
    end
    srand(seed);
end

# generate random demand values if random_demand is true, otherwise set it to default
set_demand = NamedArray(zeros(N, T), (nodes, days), ("Nodes", "Day"))
_demand = copy(demand)

if random_demand
    if random_dist == :Uniform
        set_demand[consumers, :] = 60
        set_demand[:S, :] = 250
        _demand = rand(N,T).*set_demand
    elseif random_dist == :Gaussian
        set_demand[consumers, :] = 40
        set_demand[:S, :] = 150
        _demand = max.(0, randn(N,T).*set_demand.*random_std_p + set_demand)
    else
        println("Invalid Random Distribution: ", random_dist)
        return
    end
end
    
# generate random supply values if random_supply is true, otherwise set it to default
set_supply = NamedArray(zeros(N, T), (nodes, days), ("Nodes", "Day"))
_supply = copy(supply)
    
if random_supply
    if random_dist == :Uniform
        set_supply[producers, :] = 500
        set_supply[us, :] = 300
        _supply = rand(N,T).*set_supply
    elseif random_dist == :Gaussian
        set_supply[producers, :] = 350
        set_supply[us, :] = 200
        _supply = max.(0, randn(N,T).*set_supply.*random_std_p + set_supply)
    else
        println("Invalid Random Distribution: ", random_dist)
        return
    end
    _supply[:gamma, :] = 10000;
end


# generate no wind days if specified
if random_no_wind
    no_wind = fillrand(days, no_wind)
    println("No wind days: ", no_wind)
end
    
# set no_wind days to have 0 supply from alpha and beta
if length(no_wind) > 0
    if all([i in days for i in no_wind])
        _supply[:alpha, no_wind] = 0
        _supply[:beta, no_wind] = 0
    else
        println("Error: ", [i for i in no_wind if !(i in days)], " not included in days ", days)
        println("Ignoring no wind days...")
    end
end
                        
# generate no sun days if specified
if random_no_sun
    no_sun = fillrand(days, no_sun)
    println("No sun days: ", no_sun)
end

# set no_sun days to have 0 supply from C, L, K
if length(no_sun) > 0
    if all([i in days for i in no_sun])
        _supply[us, no_sun] = 0
    else
        println("Error: ", [i for i in no_sun if !(i in days)], " not included in days ", days)
        println("Ignoring no sun days...")
    end
end
                                            
if random_supply
    println("Randomized supply: ", _supply)
end
if random_demand
    println("Randomized demand: ", _demand)
end
                                            
# calculate b matrix (net supply)
b = NamedArray(zeros(N,T), (nodes, days), ("Nodes", "Day"))
for n in nodes
    for t in days
        b[n,t] = _supply[n,t] - _demand[n,t]
    end
end
 
# create model
m = Model(solver=ClpSolver())

@variable(m, x[arcs, days] >= 0) # energy flow from node i to j
@variable(m, s[nodes, 1:max(days...)+1] >= 0) # energy stored in battery at node i on day t
@variable(m, y[arcs, days] >= 0) # cap for flow along arc (i, j)

# # useful expressions
@expression(m, losses[a in arcs, t in days], loss*distance[a]*x[a, t]) # loss along arc a on day t
@expression(m, energy_transmitted[a in arcs, t in days], x[a, t] - losses[a, t]) # energy transmitted along a on day t
@expression(m, flow_in_lossless[k in nodes, t in days], 
    sum(x[(i,j), t] for (i,j) in arcs if j == k)) # flow into node k without considering loss on day t
@expression(m, flow_in[k in nodes, t in days], 
    sum(energy_transmitted[(i,j), t] for (i,j) in arcs if j == k)) # flow into k on day t
@expression(m, flow_out[k in nodes, t in days], 
    sum(x[(i,j), t] for (i,j) in arcs if i == k)) # flow out of k on day t
@expression(m, energy_lost, sum(loss*distance[a]*x[a, t] for a in arcs for t in days) 
    + (1-eff_in*eff_out)*sum(s[k, t] for k in nodes for t in 1:days[end]+1)) # losses from transport and batteries
@expression(m, dirty_power, sum(flow_out[:gamma, t] for t in days)) # power provided by gamma (main grid)
@expression(m, net_outflow[k in nodes, t in days], flow_out[k, t] - flow_in[k, t]) # net outflow from node k on day t
# leftover power for node k on day t                                                                                                                        
@expression(m, leftover[k in nodes, t in days], b[k, t] - net_outflow[k, t] + s[k, t]) 
# total power generated by producers
@expression(m, production, sum(net_outflow[k, t] for k in producers, t in days if k != :gamma))
# sum of the flow capacities
@expression(m, capsum, sum(y))                                                                                            

# power balance for node k on day t                                                                                                                                        
@constraint(m, power_balance[k in nodes, t in days], net_outflow[k, t] + s[k, t+1] - eff_out*s[k, t] <= b[k, t])
# battery input efficiency constraint                                                                                                                                        
@constraint(m, battery_efficiency[k in nodes, t in days], s[k, t+1] <= eff_in*leftover[k, t])
# no initial battery power available                                                                                                                                        
@constraint(m, starting_battery_savings[k in nodes], s[k, days[1]] == 0)
# battery capacity constraint                                                                                                                    
@constraint(m, battery_capacity[k in nodes, t in 1:days[end]+1], s[k, t] <= cap[k])
# just provide minimum energy needed constraint                                                                                                                                        
@constraint(m, sufficiency[k in nodes, t in days], -net_outflow[k, t] <= _demand[k, t] + s[k, t])
# flow cap symmetry constraint                                     
@constraint(m, symmetry[(i,j) in arcs, t in days], y[(i,j), t] == y[(j,i), t])
# flow cap constraint                                                                                                                                        
@constraint(m, limit_flow[a in arcs, t in days], x[a,t] <= y[a,t])
# cycle breaking constraint
@constraint(m, cyclebreaker[(i,j) in arcs, t in days], x[(i,j),t] + x[(j,i),t] == y[(i,j), t])                                                                                                                                        

# no-cycle solve iterations                                                                                                                                        
cp_iter = 0;
cp_val = cp;                                                                                                                                        
has_cycles = true;

# initialize data variables outside of while-loop                                                                                                                                        
flow = flowcaps = lost = storage = revenue = dirt = costs = cyclesum = 0;                                                                                                                                        
                                                                                                                                        
while (has_cycles & (cp_iter < max_iter))
                                                                                                                                            
@objective(m, Min, cost*energy_lost + penalty*dirty_power - price*production + cp_val*capsum)
                                                                                                                                            
status = solve(m)
 
# calculate resulting data                                                                                                                                        
flow = NamedArray(zeros(length(arcs), length(days)), (arcs, days), ("Edges","Day"))
flowcaps = NamedArray(zeros(length(arcs), length(days)), (arcs, days), ("Edges","Day"))
lost = NamedArray(zeros(length(arcs), length(days)), (arcs, days), ("Edges","Day"))                                                                                                            
storage = NamedArray(zeros(length(nodes), length(days)), (nodes, days), ("Nodes","Day"))
revenue = getvalue(price*production)
dirt = getvalue(penalty*dirty_power)
costs = getvalue(cost*energy_lost)
cyclesum = getvalue(cp*capsum)
                                                                                                            
for t in days
    for a in arcs
        flow[a,t] = getvalue(x[a,t])
        lost[a,t] = getvalue(losses[a,t])
        flowcaps[a,t] = getvalue(y[a,t])
    end 
    for n in nodes
        storage[n,t] = getvalue(s[n,t])
    end
end

# determines whether cycles are present
has_cycles = (sum(flow[(i,j),t]*flow[(j,i),t] for (i,j) in arcs for t in days) > 0)

# increase iteration cycle
cp_iter += 1;

if(verbose)
    println("--- Iteration: ", cp_iter, " ---")
    println("cp=", cp_val)
else
    print(".")
end

if has_cycles
    if(verbose)
        println("Cycles detected.")
    end
    cp_val += cp_step
end

end # end of while loop

# verbose output
if(verbose)
    print("\n")
    if(!has_cycles)
        println("No-cycle solution found after ", cp_iter, " iterations. cp=", cp_val)
    end
    println("------Solution------")
    println(getvalue(x))
    println("------")
    println(getvalue(s))
    println("Total energy losses will be ", getvalue(energy_lost), " kWh.")
    println("Total dirty energy used will be ", getvalue(dirty_power), " kWh.")
    println("Revenue: \$", revenue)
end

# construct data dictionary                                                                                                                                                    
data = Dict("flows"=>flow, "storage"=>storage, "flowcaps"=>flowcaps,
    "revenue"=>revenue, "losses"=>lost, "costs"=>costs, "cycles?"=> has_cycles,
    "dirt"=>dirt, "cycleterm"=>cyclesum);
if random_demand
    data["demand"] = _demand
end
if random_supply
    data["supply"] = _supply
end                                                                                                                                                    
if random
    data["seed"] = seed
end
if cp_iter > 1
    data["cp"] = cp_val
end
                                                                                                                                                    
return data
end

solve_flows (generic function with 1 method)

In [64]:
data = solve_flows(price=1000, cost=1, penalty=10, cp=10, verbose=false,
    cp_step=100, max_iter=10,
    random_supply=true, random_demand=true, random_dist=:Uniform, random_std_p=0.5, no_sun=3, no_wind=2, seed=603204)


No wind days: [1, 7]
No sun days: [5, 3, 4]
Randomized supply: 12×7 Named Array{Float64,2}
Nodes ╲ Day │       1        2        3        4        5        6        7
────────────┼──────────────────────────────────────────────────────────────
:S          │     0.0      0.0      0.0      0.0      0.0      0.0      0.0
:K          │ 28.0056  21.7075      0.0      0.0      0.0   273.45  157.936
:P          │     0.0      0.0      0.0      0.0      0.0      0.0      0.0
:gamma      │ 10000.0  10000.0  10000.0  10000.0  10000.0  10000.0  10000.0
:L          │ 207.126  277.959      0.0      0.0      0.0  13.8369  60.7957
:B          │     0.0      0.0      0.0      0.0      0.0      0.0      0.0
:O          │     0.0      0.0      0.0      0.0      0.0      0.0      0.0
:beta       │     0.0  379.589  404.928  497.594  115.844  366.679      0.0
:C          │ 267.617  283.562      0.0      0.0      0.0  51.9477  254.608
:M          │     0.0      0.0      0.0      0.0      0.0      0.0      0

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /Users/bryan_luu30794/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /Users/bryan_luu30794/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /Users/bryan_luu30794/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /Users/bryan_luu30794/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stdout,Timer})() at ./event.jl:436
while loading In[64], in expression starting on line 1


Dict{String,Any} with 13 entries:
  "flows"     => 32×7 Named Array{Float64,2}…
  "cp"        => 210
  "flowcaps"  => 32×7 Named Array{Float64,2}…
  "revenue"   => 2.59498e6
  "supply"    => 12×7 Named Array{Float64,2}…
  "storage"   => 12×7 Named Array{Float64,2}…
  "losses"    => 32×7 Named Array{Float64,2}…
  "cycleterm" => 1.10108e5
  "demand"    => 12×7 Named Array{Float64,2}…
  "cycles?"   => false
  "seed"      => 603204
  "costs"     => 1012.3
  "dirt"      => 5340.32

In [ ]:
day = 1
plot_flow(data["flows"][:,day])

In [ ]:
for day in days
    println("====== Day ", day, " ======")
    sleep(0.1)
    display(plot_flow(data["flows"][:, day]))
end

In [ ]:
println(sum(data["supply"], 1) - 10000)
println(sum(data["demand"], 1))